In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import make_blobs
from torch.utils.data import DataLoader, TensorDataset

# Logistic regression
We will use the `make_blobs` function to generate the data

In [ ]:
X,y = make_blobs(
    n_samples=250,
    n_features=2,
    centers=[[.2, .2],[.8,.6]],
    cluster_std=[.1,.25]
)

# Convert to tensor and create dataloader
dataset = TensorDataset(torch.Tensor(X), torch.Tensor(y.reshape(-1,1)))
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

plt.scatter(X[:,0], X[:,1], c=y)

Create a class `LogisticRegression`, extending `nn.Module`. In its initialization, create a `Linear` layer ($D \mapsto 1$) and a `Sigmoid` one. You could also use two outputs and softmax. Create a forward method that applies the linear layer and then the sigmoid one. The forward method should return the output of the sigmoid layer.

In [ ]:
class LogisticRegression(nn.Module):
    def __init__(self, input_dimensions):
        super().__init__()
        ????

    def forward(self,x):
        return ???

In [ ]:
mod = LogisticRegression(input_dimensions=2)

Define the loss function (in this case binary cross entropy) and an optimizer (for example SGD)

In [ ]:
loss_fun = 0 # Select apropriate loss function e.g. nn.BCELoss()
optimizer = optim.SGD(mod.parameters(), lr=0.1)

Iterate a training step

In [ ]:
# Training loop goes here


Great! Let us have a look at the predictions:

In [ ]:
preds = mod(torch.Tensor(X)).detach().numpy()

plt.scatter(X[:,0], X[:,1], c=y, s=100)
plt.scatter(X[:,0], X[:,1], c=preds)

As you can see, the points are assigned a probability of being in class 1 (yellow) which match the original labels.

## Decision boundary
However, we may want to investigate what decisions is assigned to any given combination of the features. In this case, since the data is 2 dimensional, we can create a grid and evaluate the model on it.

In [ ]:
res = 50
grid_1D = np.linspace(-.5, 1.5, res)
xx,yy = np.meshgrid(grid_1D, grid_1D)

X_grid = np.column_stack((xx.flatten(), yy.flatten()))

plt.scatter(X_grid[:,0], X_grid[:,1],s=2)

X_grid = torch.Tensor(X_grid)

In [ ]:
preds = mod(X_grid).detach().numpy()
plt.scatter(X_grid[:,0], X_grid[:,1],c=preds, s=2)

In [ ]:
plt.contourf(grid_1D, grid_1D, preds.reshape(res,res), levels=20)
plt.colorbar()
plt.scatter(X[:,0], X[:,1], c=y, edgecolors='k')